# ViT on Fashion MNIST

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import torch
from lightning.pytorch import Trainer

from atttools import (
    FashionMNISTDataModule,
    ClassifierViT
)

In [ ]:
_ = torch.manual_seed(1223334444) # set random seed manually

## Fashion MNIST data

In [ ]:
fmnist = FashionMNISTDataModule(
    data_dir='../run/data/',
    mean=None,
    std=None,
    batch_size=32,
)

fmnist.prepare_data() # download data if not yet done
fmnist.setup(stage='test') # create test set

In [ ]:
test_loader = fmnist.test_dataloader()
x_batch, y_batch = next(iter(test_loader))

In [ ]:
class_names = fmnist.test_set.classes

fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(5, 4.5))
for idx, ax in enumerate(axes.ravel()):
    image = x_batch[idx, 0].numpy()
    ax.imshow(image, cmap='gray', vmin=0, vmax=1)
    ax.set_title(class_names[y_batch[idx]])
    ax.set(xticks=[], yticks=[], xlabel='', ylabel='')
fig.tight_layout()

## ViT model

In [ ]:
ckpt_file = '../run/fmnist/version_0/checkpoints/best.ckpt'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

vit = ClassifierViT.load_from_checkpoint(ckpt_file)

vit = vit.eval()
vit = vit.to(device)

## Testing

In [ ]:
trainer = Trainer(logger=False)

test_metrics = trainer.test(
    model=vit,
    dataloaders=test_loader,
    verbose=True
)

In [ ]:
confmat = vit.test_confmat.compute().numpy() # assemble confusion matrix
norm_confmat = confmat / confmat.sum(axis=1).reshape(-1, 1) # normalize rows

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
img = ax.imshow(
    norm_confmat, cmap='viridis', aspect='equal', vmin=0, vmax=1
)
ax.set(
    xlabel='predicted',
    ylabel='actual',
    xticks=list(range(len(class_names))),
    yticks=list(range(len(class_names)))
)
ax.set_xticklabels(class_names, rotation='vertical')
ax.set_yticklabels(class_names, rotation='horizontal')
fig.colorbar(img)
fig.tight_layout()